# **Group 19 Project**  **GIVE BETTER TITLE**

# **Introduction**

Video games have evolved from simple pastimes into complex environments that offer rich data about user behavior and interaction. This report is grounded in a real-world data science project conducted by a research group in Computer Science at UBC, led by Frank Wood. The team has established a Minecraft server where every action taken by players is recorded. By capturing this data, the researchers aim to unlock insights into how individuals navigate and interact within virtual worlds.

The project has multiple objectives, and we will focus on understanding the characteristics and behaviors that most predict a player's likelihood to subscribe to a game-related newsletter. This targeted approach helps ensure that sufficient resources—such as software licenses and server hardware—are available to support the anticipated influx of players. By investigating player behavior through detailed analytics, the study aims to inform future strategies for engagement, recruitment, and resource allocation in online gaming communities. This report will detail the specific methodologies used to analyze the player data, the key findings related to newsletter subscription behavior, and the implications of these findings.

# **Question**

Are age and gender predictive of subscription status to a game related newsletter in the player.csv data set?

## Data Set Description

The "players.csv" dataset contains observations collected for multiple different variables from people who played on the MineCraft server. The data frame contains 7 variables and 196 rows of data, producing 1372 observations in total. The variables are ordered in the table left to right are:

- `Experience`
    - This variable describes the level at which each player is at in terms of playing the game. They are categorized into 5 categories: Amateur, Beginner, Regular, Pro, and Veteran
    - In the data set, this variable is a character type 
- `Subscribe`
    - This variable describes whether or not the player is subscribed to a game-related newsletter. They are categorized into 2 categories: True or False
    - In the data set, this variable is a logical type 
- `Hashed Email`
    - This variable describes lists each players email in a hashed format. 
    - In the data set, this variable is a character type 
- `Hours Played`
    - This variable describeshow many hours each player spent playing the game (in hours). 
    - In the data set, this variable is a double type, so the values can contain decimals 
- `Name`
    - This variable states the players first name
    - In the data set, this variable is a character type 
- `Gender`
    - This variable describes the gender of each player. 
    - In the data set, this variable is a character type
- `Age`
    - This variable describes the age of the players (in years) 
    - In the data set, this variable is an integer type, so the values are whole numbers only

**This is the data set that will be used in the analysis**

The "sessions.csv" data contains observations collected for multiple different variables from people who played on the MineCraft server. The data frame contains 5 variables and 1535 rows of data, producing 7675 observations in total. The variables are ordered in the table from left to right are:

- `hashedEmail`
    - This variable gives a string of letters and numbers that represent the players email address. 
    - In the data set, this variable is a character type 
- `start_time`
    - This variable gives the exact date (DD/MM/YR) and time (24 hour clock) that the player started their session.
    - In the data set, this variable is a character type 
- `end_time`
    - This variable the exact date (DD/MM/YR) and time (24 hour clock) that the player ended their session. 
    - In the data set, this variable is a character type 
- `original_start_time`
    - This variable describes the original start time of players  (**IDK HOW TO DESCRIBE IT BETTER**)
    - In the data set, this variable is a double type, so the values can contain decimals 
- `original_end_time`
    - This variable describes the original end time of players (**SAME HERE**)
    - In the data set, this variable is a double type, so the values can contain decimals 


This data will not be used in the analysis

In [5]:
library(tidyverse)
library(rvest)
library(dplyr)

In [9]:
url_sessions <- "https://raw.githubusercontent.com/IFQXK/DSCI-100-project-group-19/refs/heads/main/sessions.csv"
sessions_data <- read.csv(url_sessions)
head(sessions_data)

url_players <- "https://raw.githubusercontent.com/IFQXK/DSCI-100-project-group-19/refs/heads/main/players.csv"
players_data <- read.csv(url_players)
head(players_data)

,hashedEmail,start_time,end_time,original_start_time,original_end_time
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
2,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
3,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
4,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
5,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
6,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


,experience,subscribe,hashedEmail,played_hours,name,gender,Age
,<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<int>
1,Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
2,Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
3,Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
4,Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
5,Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
6,Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17
